<a href="https://colab.research.google.com/github/EtzionR/LM4GeoAI/blob/main/Geo_LLM_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex 3: Geo LLM Agents

### created by Etzion Harari | Geo-AI Course

[**https://github.com/EtzionR/LM4GeoAI**](https://github.com/EtzionR/LM4GeoAI)

In [1]:
!pip install cohere
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.2 MB/s eta 0:00:00


In [2]:
from difflib import get_close_matches
from getpass import getpass

import osmnx as ox
import folium
import cohere

Cohere models: [https://docs.cohere.com/docs/models#command](https://docs.cohere.com/docs/models#command)

Cohere MCP: [https://docs.cohere.com/page/basic-tool-use](https://docs.cohere.com/page/basic-tool-use)

In [3]:
MODEL  = "command-a-03-2025"
TEMP = .01

Get your Cohere API Key:

[https://dashboard.cohere.com/api-keys](https://dashboard.cohere.com/api-keys)

In [4]:
KEY = getpass("Please Enter COHERE KEY:\n")

len(KEY)

Please Enter COHERE KEY:
··········


40

In [5]:
client = cohere.Client(api_key=KEY)
client

In [6]:
QUERY = 'What is panda bear?'

output = client.chat(model=MODEL,
                     message=QUERY,
                     temperature=TEMP).text

print(output)

A panda bear, often simply referred to as a panda, is a species of bear native to central China. Specifically, it is the **giant panda** (*Ailuropoda melanoleuca*), which is one of the most recognizable and beloved animals in the world. Here are some key facts about panda bears:

1. **Appearance**: Pandas are known for their distinctive black and white fur. They have black patches around their eyes, ears, and across their bodies, with the rest of their fur being white.

2. **Diet**: Despite being classified as carnivores, pandas primarily eat bamboo, which makes up about 99% of their diet. They also occasionally eat other vegetation, small animals, and fish.

3. **Habitat**: Pandas live in the temperate forests of the mountains in central China, primarily in Sichuan, Shaanxi, and Gansu provinces. They prefer areas with dense bamboo growth.

4. **Behavior**: Pandas are generally solitary animals, except during the brief mating season. They are excellent climbers and spend much of their 

In [7]:


def get_place_info(location_adress, place_type, radius=500):

    out = ox.features_from_address(location_adress, {"amenity": True}, radius)[['name','amenity','geometry']].rename(columns={'amenity':'place_type'})

    place_type = get_close_matches(place_type, [*out.place_type.unique()], n=1, cutoff=.0)[0]

    return out[out.place_type==place_type]



location_adress = 'מגדלי הארבעה'
place_type = 'fual'

get_place_info(location_adress, place_type)


name place_type                   geometry
element id                                                        
node    339475532         פז       fuel  POINT (34.78253 32.06808)
        714371935       סד"ש       fuel  POINT (34.78319 32.06895)
        1823251323     סונול       fuel  POINT (34.78538 32.06639)
        4990104706  דור אלון       fuel   POINT (34.78719 32.0661)

In [8]:
system_prompt = """You are an AI assistant with access to get_place_info tool, which enable you to run osm query """

tools = [
    {
        "name": "get_place_info",
        "description": "can return every type of geographic location near some given adress",
        "parameter_definitions": {
            "location_adress": {
                "description": "free text location adress",
                "type": "str",
                "required": True
            },
            "place_type": {
                "description": "type of point of interest (POI)",
                "type": "str",
                "required": True
            },
            "radius": {
                "description": "radius of search in meters",
                "type": "float",
                "required": False
            }
        }
    }]

function_map = {get_place_info.__name__:get_place_info}

In [9]:
HEB = 'מגדלי הארבעה'

user_prompt = f"I want you to return every caffe near {HEB}"


response = client.chat(
    model=MODEL,
    message=user_prompt,
    preamble=system_prompt,
    tools=tools
)

raw = response.text
print("MODEL RAW:", raw)

MODEL RAW: I will use the get_place_info tool to find every caffe near מגדלי הארבעה.


In [10]:
response.tool_calls

[ToolCall(name='get_place_info', parameters={'location_adress': 'מגדלי הארבעה', 'place_type': 'caffe'})]

In [13]:
out = function_map[response.tool_calls[0].name](**response.tool_calls[0].parameters)

out

name place_type                   geometry
element id                                                             
node    553022802        קפה שרגא       cafe    POINT (34.7903 32.0707)
        635130301          משוגעת       cafe  POINT (34.78211 32.07238)
        1419068446        Arcaffe       cafe  POINT (34.78638 32.07048)
        1438109990         רולדין       cafe     POINT (34.782 32.0742)
        1450724665           מטרו       cafe  POINT (34.78436 32.06637)
        3465156312         רולדין       cafe  POINT (34.78693 32.07173)
        3731754512          ארומה       cafe  POINT (34.78587 32.07026)
        4587267164         Stampa       cafe  POINT (34.78248 32.07329)
        6005369587      Cafeteria       cafe  POINT (34.78438 32.06917)
        6389067485      Chop Chop       cafe   POINT (34.78199 32.0737)
        6644165508          Aroma       cafe  POINT (34.78769 32.07308)
        7211406506    Max Brenner       cafe   POINT (34.78677 32.0715)
        7524489151       דובנוב 8       cafe  POINT (34.78393 32.07444)
        9245806417         לנדוור       cafe  POINT (34.78636 32.07189)
        10279238209  מאפיית לחמים       cafe  POINT (34.78436 32.06964)
        10812416453        BEITEA       cafe  POINT (34.78798 32.07223)
        10948700190     LU coffee       cafe  POINT (34.79016 32.07017)
        11691761924  Greenkitchen       cafe  POINT (34.78354 32.06934)
        12032015683           mae       cafe  POINT (34.78829 32.07195)
        12050619149         ארומה       cafe   POINT (34.78341 32.0663)
        12139988888      קפה עוטף       cafe  POINT (34.78739 32.07308)
        12635329119        רולדין       cafe   POINT (34.79232 32.0743)
        13338979218          Biga       cafe  POINT (34.78691 32.07308)
        13339015829    Bagel Cafe       cafe  POINT (34.78529 32.07127)

In [17]:
fmap = folium.Map(location=[out.geometry.y.median(), out.geometry.x.median()],
                  zoom_start=15.5, width=500, height=500)

folium.GeoJson(out).add_to(fmap)

fmap

In [ ]:
co.chat(
    model=MODEL,
    message='who is donald trump?',
    preamble=system_prompt
)

In [ ]:
import json
import cohere

# Initialize Cohere client
co = client#cohere.Client("YOUR_COHERE_API_KEY")


# ---------------------------
# TOOL IMPLEMENTATION
# ---------------------------
def calculator(operation, a, b):
    print('ACTIAVTE!')
    if operation == "add":
        return a + b
    elif operation == "mul":
        return a * b
    else:
        return "Unknown operation"


# ---------------------------
# SEND INITIAL REQUEST TO LLM
# ---------------------------
system_prompt = """
You are an AI assistant with access to tools.

Available tool:
1. calculator
   - description: simple calculator
   - args: { "operation": "add|mul", "a": number, "b": number }

If you want to use a tool, respond ONLY in the following JSON format. Dont return anything else (just the json):

{
  "tool": "calculator",
  "arguments": {
      "operation": "...",
      "a": ...,
      "b": ...
  }
}

If you can answer without a tool, answer normally.
"""

user_prompt = "What is 12.123 * 7.456?"


response = co.chat(
    model=MODEL,
    message=user_prompt,
    preamble=system_prompt
)

raw = response.text
print("MODEL RAW:", raw)

# ---------------------------
# TRY TO PARSE TOOL CALL
# ---------------------------
try:
    data = json.loads(raw)
    if data.get("tool") == "calculator":
        args = data["arguments"]
        result = calculator(args["operation"], args["a"], args["b"])
        print("TOOL RESULT:", result)

        # Send result back to model
        follow = co.chat(
            model=MODEL,
            preamble=system_prompt,
            message=f"The tool returned the answer: {result}. Now answer the user with natural language."
        )

        print("\nFINAL ANSWER:")
        print(follow.text)

    else:
        print("No tool used.")
except json.JSONDecodeError:
    print("Model did NOT output JSON. Final answer:")
    print(raw)

In [ ]:
# Define the tools

co = client

import json

def list_calendar_events(date: str):
  events = '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'
  print(f"Listing events: {events}")
  return events

def create_calendar_event(date: str, time: str, duration: int):
  print(f"Creating a {duration} hour long event at {time} on {date}")
  return True

list_calendar_events_tool = {
  "name": "list_calendar_events",
  "description": "returns a list of calendar events for the specified date, including the start time and end time for each event",
  "parameter_definitions": {
    "date": {
      "description": "the date to list events for, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    }
  }
}

create_calendar_event_tool = {
  "name": "create_calendar_event_tool",
  "description": "creates a calendar event of the specified duration at the specified time and date",
  "parameter_definitions": {
    "date": {
      "description": "the date on which the event starts, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    },
    "time": {
      "description": "the time of the event, formatted using 24h military time formatting",
      "type": "str",
      "required": True
    },
    "duration": {
      "description": "the number of hours the event lasts for",
      "type": "float",
      "required": True
    }
  }
}

# helper function for routing to the correct tool
def invoke_tool(tool_call: cohere.ToolCall):
  if tool_call.name == list_calendar_events_tool["name"]:
    date = tool_call.parameters["date"]
    return [{
        "events": list_calendar_events(date)
    }]
  elif tool_call.name == create_calendar_event_tool["name"]:
    date = tool_call.parameters["date"]
    time = tool_call.parameters["time"]
    duration = tool_call.parameters["duration"]

    return [{
        "is_success": create_calendar_event(date, time, duration)
    }]
  else:
    raise f"Unknown tool name '{tool_call.name}'"


In [ ]:
# Check what tools the model wants to use and how to use them
res = co.chat(
    model="command-a-03-2025",
    preamble="Today is Thursday, may 23, 2024",
    message="book an hour long appointment for the first available free slot after 3pm",
    force_single_step=False,
    tools=[list_calendar_events_tool, create_calendar_event_tool])
while res.tool_calls:
  print(res.text) # This will be an observation and a plan with next steps
  # invoke the recommended tools
  tool_results = []
  for call in res.tool_calls:
    tool_results.append({"call": call, "outputs": invoke_tool(call)})
  # send back the tool results
  res = co.chat(
    model="command-a-03-2025",
    chat_history=res.chat_history,
    message="",
    force_single_step=False,
    tools=[list_calendar_events_tool, create_calendar_event_tool],
    tool_results=tool_results,
  )
print(res.text) # print the final answer

In [ ]:
print(res.tool_calls)